<a href="https://colab.research.google.com/github/kani91/ProgrammingAssignment2/blob/master/8_constructing_cnn_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Model

### Introduction

In this lesson, we'll work through constructing a convolutional neural network in Pytorch, and understanding the transformations that occur in our different layers.  We'll start by interpreting a premade neural network, and then we'll move onto constructing our own neural network let's get started.

### Loading our Data

To begin, we'll set the device as `cuda` so that we can perform calculations on the GPU and thus speed up training time.

In [ ]:
import torch
torch.device("cuda")

device(type='cuda')

Let's begin by loading our Fashion MNIST dataset.

In [ ]:
from torchvision import datasets, transforms
train = datasets.FashionMNIST("", train = True, download = True,
                       transform = transforms.Compose([transforms.ToTensor()]))
test = datasets.FashionMNIST("", train = False, download = True,
                       transform = transforms.Compose([transforms.ToTensor()]))
X_train_reshaped = train.data.reshape(-1, 100, 1, 28, 28)

X_train_reshaped.shape

Extracting FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/raw




Extracting FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


torch.Size([600, 100, 1, 28, 28])

In [ ]:
y_reshaped = train.targets.reshape(-1, 100)
y_reshaped.shape

torch.Size([600, 100])

In [ ]:
combined = list(zip(X_train_reshaped, y_reshaped))

### Building a Neural Network

Now this is the neural network we worked in the previous lesson.

```python
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)

  def forward(self, t):
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)
    # conv 2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)
    t = t.reshape(-1, 12*4*4)
    t = self.fc1(t)
    t = F.relu(t)
    t = self.fc2(t)
    t = F.relu(t)
    t = self.out(t)
    return F.log_softmax(t, dim = 1)
```

### Improving upon our network

Now let's begin to define our own neural network.  

1. We'll create two Conv2d layers, both with a stride of 1.  

* The first convolutional layer should have kernels of size 5, and
* The second convolutional layer should have kernels of size 3.  
* The first layer should have 6 kernels and the second layer should have 12 kernels.

Define the neural network below, and check that it has these parameters.

In [ ]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, stride=1, kernel_size=5, padding = 2)
        self.conv2 = nn.Conv2d(6, 12, stride=1, kernel_size=3, padding = 1)

In [ ]:
Net()

# Net(
#   (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
#   (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
# )

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

Now update the padding in our neural network.  A good rule of thumb is that the padding should be half the dimensions of kernels, rounding down.  So that leaves us with a padding of 2 for the first layer, and 1 for the second.

In [ ]:
5//2, 3//2

(2, 1)

> Check that the output matches that below.

In [ ]:
Net()

# Net(
#   (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
#   (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
# )

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

Now copy the neural network into the cell below.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, stride=1, kernel_size=5, padding = 2)
        self.conv2 = nn.Conv2d(6, 12, stride=1, kernel_size=3, padding = 1)
        
    def forward(self, X):
        A1 = F.relu(self.conv1(X))
        pooled_1 = F.avg_pool2d(A1, kernel_size = 2)
        A2 = F.relu(self.conv2(pooled_1))
        pooled_2 = F.avg_pool2d(A2, kernel_size = 2) # 16x2x2
        return pooled_2

And now let's move over to beginning our forward function.  This is where the activation and sampling of our convolutional layers will occur.

For each of the two convolutional layers, apply the following sequence.

* `Conv > Relu > AvgPool`

> Each `pool2d` should have a kernel size of 2.

We can check the output from this forward function if we `return` the last line from forward (the result of our second average pooling).  Then, we'll initialize the neural network and check the results.

In [ ]:
net_with_forward = Net()

In [ ]:
first_batch = X_train_reshaped[0].float()

In [ ]:
outputs_second_pooling = net_with_forward(first_batch)

outputs_second_pooling.shape

torch.Size([100, 12, 7, 7])

In [ ]:
outputs_second_pooling.reshape(-1, 12*7*7)

tensor([[0.0000e+00, 0.0000e+00, 9.5723e-01,  ..., 0.0000e+00, 0.0000e+00,
         8.3274e-01],
        [6.0529e+00, 3.2207e+01, 4.9118e+01,  ..., 1.9571e-02, 1.0181e+01,
         2.1316e+00],
        [0.0000e+00, 3.5907e+00, 1.7251e+01,  ..., 2.1303e+00, 4.2261e+00,
         4.6745e-02],
        ...,
        [0.0000e+00, 2.0090e+00, 2.8819e+01,  ..., 4.9750e+00, 7.5619e+00,
         7.0089e-02],
        [5.5269e-02, 9.1057e+00, 5.2705e+01,  ..., 1.9983e+00, 6.9033e+00,
         7.0089e-02],
        [1.4377e-02, 8.4065e-01, 6.7479e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<ViewBackward>)

So we now have an output for each observation of 12 kernels each of size 7 by 7.  Let's see if we can unpack why.  

1. `Conv(1, 6, stride=1, kernel_size=5, padding = 2) > Relu > AvgPool(2)`
2. `Conv(6, 12, stride=1, kernel_size=3, padding = 1) > Relu > AvgPool(2)`

Once again, this is our formula for calculating the output dimensions from each layer.

$output = \frac{i - k + 2p}{s} + 1$ 

And remember we start with a $28x28$ image.

> Calculate the output dimenion from first convolutional sequence (that ends with avgpooling).

In [ ]:
(28 - 5 + 2*2)/2 + 1 

# 14.5 -> 14

14.5

> Then calculate the output dimension from the second convolutional sequence.

In [ ]:
(14 - 3 + 2*1)/2 + 1

# 7.5 -> 7

7.5

Ok, now let's finish up the neural network.  Copy the neural network that we have so far into the cell below.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, stride=1, kernel_size=5, padding = 2)
        self.conv2 = nn.Conv2d(6, 12, stride=1, kernel_size=3, padding = 1)
        
    def forward(self, X):
        A1 = F.relu(self.conv1(X))
        pooled_1 = F.avg_pool2d(A1, kernel_size = 2)
        A2 = F.relu(self.conv2(pooled_1))
        pooled_2 = F.avg_pool2d(A2, kernel_size = 2) # 16x2x2
        return pooled_2

> Then check the shape of the return value from the pooled output.

In [ ]:
net = Net()

In [ ]:
pooled_output = net(first_batch)
pooled_output.shape

torch.Size([100, 12, 7, 7])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, stride=1, kernel_size=5, padding = 2)
        self.conv2 = nn.Conv2d(6, 12, stride=1, kernel_size=3, padding = 1)
        self.L1 = nn.Linear(7*7*12, out_features=64)
        self.L2 = nn.Linear(64, out_features=10)
        
    def forward(self, X):
        A1 = F.relu(self.conv1(X))
        pooled_1 = F.avg_pool2d(A1, kernel_size = 2)
        A2 = F.relu(self.conv2(pooled_1))
        pooled_2 = F.avg_pool2d(A2, kernel_size = 2) # 16x2x2
        reshaped_pool = pooled_2.reshape(-1, 7*7*12)
        A3 = F.relu(self.L1(reshaped_pool))
        A4 = self.L2(A3)
        return F.log_softmax(A4, dim = 1)

Then add two linear layers to the neural network.  To do so, we'll need to perform the following steps.

* To pass the output from our second pooling to a linear layer, we'll first need to reshape the data from each output into a vector.  Perform the proper reshaping in the forward method.

* Then, define the two new linear layers.  The first linear layer should have 64 neurons.

* And in the forward method let's continue using Relu as our activation function, and for the second linear layer use `log_softmax` for the activation function.

To check that we have things properly setup, let's pass through another batch of data, and make sure there are no errors.

In [ ]:
net_with_linear = Net()

In [ ]:
first_batch.shape

torch.Size([100, 1, 28, 28])

In [ ]:
net_with_linear(first_batch).shape

# torch.Size([100, 10])

torch.Size([100, 10])

We should see that there are now 10 outputs for each of the 100 observations in the batch.  Then let's move onto performing training.

### Training our Network

Below using a learning rate of `.0005` and 15 epochs to perform training.

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=.0005)

x_loss = nn.CrossEntropyLoss()

Then fill in the code for training loop below.

In [ ]:
for epoch in range(15):
    for X_batch, y_batch in combined:
        net.zero_grad()
        preds = net(X_batch.float())
        loss = F.cross_entropy(preds, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)

RuntimeError: ignored

After the network has finished training, let's make predictions with the network on the test data.

In [ ]:
test.data.shape

torch.Size([10000, 28, 28])

Reshape the test data so that there is a dimension for the channel.

Then we can make predictions with the test data and our `net_with_linear` neural network.

In [ ]:
predictions = net_with_linear(test_X_channel.float())

NameError: ignored

Then calculate the argmax of each prediction.

In [ ]:
max_predictions = torch.argmax(predictions, dim = 1)

NameError: ignored

And then use sklearn to calculate the accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_y, max_predictions)
# 0.8934

NameError: ignored

### Summary 

In this lesson, we constructed our own convolutional neural network from scratch.  We did so using the following architecture.

We initialized two convolutional layers, where each convolutional layer followed the following sequence.

1. `Conv > Relu > AvgPool`
2. `Conv > Relu > AvgPool`

Then we defined two linear layers with the following sequences: 
3. `L1 > Relu`
4. `L2 > log_softmax`

### Resources

[Fashion MNist](https://towardsdatascience.com/build-a-fashion-mnist-cnn-pytorch-style-efb297e22582)